In [1]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

temperature value --> how creative  we want  our model to be

0 --> temperature it means model is very safe it is not taking any bets.

1 --> it  will take risk it might generate wrong output ut it is very creative.

In [4]:
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature=0.6)
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.6, "max_length": 256})

In [6]:
text = "What is the capital of sri lanka"

#print(llm.predict(text))
print(llm_huggingface.predict(text))

Colombo


Prompt Template & LLM Chain

In [8]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=["country"], 
                               template="Tell me the capital of {country}")

prompt_template.format(country="srilanka")

'Tell me the capital of srilanka'

In [12]:
from langchain.chains import LLMChain
#llm_huggingface.predict(prompt_template=prompt_template, text="srilanka") -> whithou chain it will give an error
chain=LLMChain(llm=llm_huggingface, prompt=prompt_template)
chain.run("srilanka")

d:\LangChain\LangChain\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Colombo'

Combining multipe chains using simple sequential chain


In [14]:
capital_prompt=PromptTemplate(input_variables=["country"],
                              template="Please tell me the capital of {country}")

capital_chain=LLMChain(llm=llm_huggingface, prompt=capital_prompt)


famous_prompt=PromptTemplate(input_variables=["capital"],
                             template="Suggest me some amazing places to visit in {capital}")


In [15]:
famous_chain=LLMChain(llm=llm_huggingface, prompt=famous_prompt)

In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("srilanka")

'The Royal Palace, the Royal Museum, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, th

## Sequential chain

In [20]:
capital_prompt=PromptTemplate(input_variables=["country"],
                              template="Please tell me the capital of {country}")

capital_chain=LLMChain(llm=llm_huggingface, prompt=capital_prompt, output_key="capital")

famous_prompt=PromptTemplate(input_variables=["capital"],
                             template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm_huggingface, prompt=famous_prompt, output_key="places")

In [22]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain, famous_chain], 
                        input_variables=['country'],
                        output_variables=['capital', 'places'])

chain({"country": "srilanka"})

d:\LangChain\LangChain\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'country': 'srilanka',
 'capital': 'Colombo',
 'places': 'The Royal Palace, the Royal Museum, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Royal Museum of Sri Lanka, the Ro

## Chatmodels with ChatOpenAI

In [23]:
from langchain.chat_models import ChatOpenAI, google_palm

In [24]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [32]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature=0.6, model='gpt-3.5-turbo')
chatllm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.6, "max_length": 256})

In [1]:
# chatllm

In [33]:
# chatllm([
#     SystemMessage(content="You are a comedian AI assistant"),
#     HumanMessage(content="Please provide some comedy punchline on AI")
# ])

In [35]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Specify the model you want to use
model_name = "meta-llama/Meta-Llama-3-8B"  # or another model like "EleutherAI/gpt-neo-1.3B"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to generate a response
def generate_response(prompt, max_length=100, temperature=0.6):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, temperature=temperature, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
prompt = "Tell me a joke about artificial intelligence."
response = generate_response(prompt)
print(response)


KeyboardInterrupt: 

## Prompt Template + LLM + Output parsers

In [36]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [37]:
class Cummaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [38]:
template="You are a helpful assistant. When the user give any input, you should generate 5 words synonyms in a cumma seperated list"

human_template="{text}"

chatprompt=ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [ ]:
chain = chatprompt | chatllm | Cummaseparatedoutput()

In [39]:
chain.invoke({"text": "intelligent"})

ValueError: Missing some input keys: {'country'}